In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv(r"C:\Users\praka\Desktop\Studies\AmEXkaggle\amex-default-prediction\train_data.csv")

In [3]:
df_customer = df_train.groupby('customer_ID')

In [4]:
df_customer

In [5]:
df_final_0 = df_customer.last()

In [6]:
df_final_0.shape

(458913, 189)

In [7]:
# Reading the train labels
train_labels = pd.read_csv(r"C:\Users\praka\Desktop\Studies\AmEXkaggle\amex-default-prediction\train_labels.csv")

In [8]:
train_comb_0 = pd.merge(df_final_0, train_labels, on='customer_ID')

In [9]:
train_comb_0

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,...,NaN,NaN,0.007186,0.004234,0.005086,NaN,0.005810,0.002970,0.008533,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,...,NaN,NaN,0.002980,0.007479,0.007870,NaN,0.003284,0.003169,0.008514,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0.009704,0.004284,0.812650,0.006450,NaN,0.003796,0.007196,...,NaN,NaN,0.007383,0.006623,0.000964,NaN,0.002202,0.000834,0.003444,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,...,NaN,NaN,0.002704,0.006184,0.001899,NaN,0.008183,0.005560,0.002983,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0.005573,0.007679,0.815746,0.001247,0.176403,0.000231,0.005528,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008154,0.006944,0.000905,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,2018-03-31,0.844229,0.447585,0.028515,1.009867,0.001928,0.128707,0.003482,0.005893,...,NaN,NaN,0.007917,0.001520,0.005530,NaN,0.001430,0.003009,0.004843,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,2018-03-22,0.831279,0.033670,0.292360,0.055656,0.006953,0.194697,0.005791,0.233078,...,NaN,NaN,0.004393,0.006185,0.001480,NaN,0.004794,0.009230,0.006435,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,2018-03-07,0.800522,0.267018,0.020563,1.007023,0.000957,0.066648,0.007424,0.006314,...,NaN,NaN,0.006035,0.002869,0.002149,NaN,0.008031,0.000340,0.002148,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,2018-03-23,0.754129,0.008619,0.015838,0.714486,0.000993,0.408849,0.003392,0.050048,...,NaN,NaN,1.009894,0.004478,0.949723,0.446255,1.009310,0.002502,0.185527,1


In [15]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OneHotEncoder

In [16]:
train_comb_0['D_63'] = train_comb_0['D_63'].astype('category')
train_comb_0['D_64'] = train_comb_0['D_64'].astype('category')

In [17]:
train_comb_0['D_63_new'] = train_comb_0['D_63'].cat.codes
train_comb_0['D_64_new'] = train_comb_0['D_64'].cat.codes

In [18]:
enc=OneHotEncoder()
enc_data=pd.DataFrame(enc.fit_transform(train_comb_0[['D_63_new','D_64_new']]).toarray())

In [19]:
train_comb_0 = train_comb_0.join(enc_data)

In [20]:
train_comb_0['S_2'] = pd.to_datetime(train_comb_0['S_2'])

In [21]:
for col in train_comb_0.columns:
    print(f"{col} \t {train_comb_0[col].dtype}")

customer_ID 	 object
S_2 	 datetime64[ns]
P_2 	 float64
D_39 	 float64
B_1 	 float64
B_2 	 float64
R_1 	 float64
S_3 	 float64
D_41 	 float64
B_3 	 float64
D_42 	 float64
D_43 	 float64
D_44 	 float64
B_4 	 float64
D_45 	 float64
B_5 	 float64
R_2 	 float64
D_46 	 float64
D_47 	 float64
D_48 	 float64
D_49 	 float64
B_6 	 float64
B_7 	 float64
B_8 	 float64
D_50 	 float64
D_51 	 float64
B_9 	 float64
R_3 	 float64
D_52 	 float64
P_3 	 float64
B_10 	 float64
D_53 	 float64
S_5 	 float64
B_11 	 float64
S_6 	 float64
D_54 	 float64
R_4 	 float64
S_7 	 float64
B_12 	 float64
S_8 	 float64
D_55 	 float64
D_56 	 float64
B_13 	 float64
R_5 	 float64
D_58 	 float64
S_9 	 float64
B_14 	 float64
D_59 	 float64
D_60 	 float64
D_61 	 float64
B_15 	 float64
S_11 	 float64
D_62 	 float64
D_63 	 category
D_64 	 category
D_65 	 float64
B_16 	 float64
B_17 	 float64
B_18 	 float64
B_19 	 float64
D_66 	 float64
B_20 	 float64
D_68 	 float64
S_12 	 float64
R_6 	 float64
S_13 	 float64
B_21 	 float64
D_69

In [22]:
train_comb_0.shape

(458913, 204)

In [27]:
train_comb_0.drop(['D_63', 'D_64'], axis=1, inplace=True)

In [32]:
train_comb_0

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,1,2,3,4,5,6,7,8,9,10
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0.009704,0.004284,0.812650,0.006450,NaN,0.003796,0.007196,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0.005573,0.007679,0.815746,0.001247,0.176403,0.000231,0.005528,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,2018-03-31,0.844229,0.447585,0.028515,1.009867,0.001928,0.128707,0.003482,0.005893,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,2018-03-22,0.831279,0.033670,0.292360,0.055656,0.006953,0.194697,0.005791,0.233078,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,2018-03-07,0.800522,0.267018,0.020563,1.007023,0.000957,0.066648,0.007424,0.006314,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,2018-03-23,0.754129,0.008619,0.015838,0.714486,0.000993,0.408849,0.003392,0.050048,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [39]:
y_train = train_comb_0['target']
x_train = train_comb_0.drop(columns = ['target', 'customer_ID', 'S_2'], axis = 1)

In [40]:
model = LGBMClassifier()

In [41]:
model.fit(x_train, y_train)

LightGBMError: bad allocation